# Домашняя работа

В этой работе вам предстоит с помощью encoder-decoder архитектуры, пробуя различные ее реализации, решить задачу машинного перевода.

#### Наша задача - сделать свой собственный переводчик!

Пока что только русско-английский:) Будем учиться на текстах описания отелей, так что при успешном выполнении этого задания у вас не возникнет проблем с выбором места для остановки в путешествии, так как все отзывы вам будут высококлассно переведены!

Что необходимо обсудить до начала работы?

В наших текстах очень много редких и очень мало встречаемых слов (в каждом отеле есть своя фишка: какой-то предмет декорации или услуга, которая описывается своим словом, которое только там и встречается). Если мы будем кодировать все слова, то размер нашего словаря будет очень-очень большим.

Чтобы эффективно решить эту проблему, мы будем использовать [Byte Pair Encoding](https://github.com/rsennrich/subword-nmt) известный как __BPE__

Этот алгоритм стартует с посимвольного уровня и итеративно мерджит самые встречаемые пары. И так N итераций. На выходе мы получаем самые частые последовательности символов из которых формируются слова!

Скачайте файлы vocab.py и данные data.txt:

https://disk.yandex.ru/d/rM4Cxcw_9OUHnw

https://disk.yandex.ru/d/EvAXHib82OExQA

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

PATH = '/content/drive/MyDrive/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


BPE - очень популярный и частоиспользуемый алгоритм в задачах NLP, поэтому есть много открытых реализаций этого алгоритма

Тем кому очень интересно, как же все работает - заходите в файл vocab.py, очень советуем!

In [ ]:
!pip install subword_nmt

In [ ]:
from nltk.tokenize import WordPunctTokenizer
from subword_nmt.learn_bpe import learn_bpe
from subword_nmt.apply_bpe import BPE

tokenizer = WordPunctTokenizer()
def tokenize(x):
    return ' '.join(tokenizer.tokenize(x.lower()))

# разбиваем и токенизируем тексты, записываем обработанные токены в файл
with open('train.en', 'w') as f_src,  open('train.ru', 'w') as f_dst:
    for line in open(PATH + 'data.txt'):
        src_line, dst_line = line.strip().split('\t')
        f_src.write(tokenize(src_line) + '\n')
        f_dst.write(tokenize(dst_line) + '\n')

# строим и применяем bpe кодирование
bpe = {}
for lang in ['en', 'ru']:
    learn_bpe(open('./train.' + lang), open('bpe_rules.' + lang, 'w'), num_symbols=8000)
    bpe[lang] = BPE(open('./bpe_rules.' + lang))

    with open('train.bpe.' + lang, 'w') as f_out:
        for line in open('train.' + lang):
            f_out.write(bpe[lang].process_line(line.strip()) + '\n')

100%|██████████| 8000/8000 [00:09<00:00, 847.27it/s] 


### Построение словарей, разбиение данных

Сейчас, когда мы обучили BPE алгоритм на наших данных, построим словарь соответствия токена и его индекса, чтобы нам было затем удобно смотреть переводы и переводить новые предложения

Также сделаем разбиение на train/test

In [ ]:
import numpy as np

In [ ]:
data_inp = np.array(open('./train.bpe.ru').read().split('\n'))
data_out = np.array(open('./train.bpe.en').read().split('\n'))

from sklearn.model_selection import train_test_split
train_inp, dev_inp, train_out, dev_out = train_test_split(data_inp, data_out, test_size=3000,
                                                          random_state=42)
for i in range(3):
    print('inp:', train_inp[i])
    print('out:', train_out[i], end='\n\n')

inp: на территории обустроена бесплатная частная парковка .
out: free private parking is available on site .

inp: кроме того , в 5 минутах ходьбы работают многочисленные бары и рестораны .
out: guests can find many bars and restaurants within a 5 - minute walk .

inp: отель san mi@@ gu@@ el расположен в центре мор@@ ели@@ и , в 750 метрах от главной площади города и кафедрального собора .
out: hotel san miguel is located in central more@@ lia , 750 metres from the city ’ s main square and cathedral .



In [ ]:
import os, sys
vocab_path = os.path.join(PATH, "vocab.py")
assert os.path.isfile(vocab_path), f"Нет файла: {vocab_path}"

if PATH not in sys.path:
    sys.path.insert(0, PATH)

from vocab import Vocab
inp_voc = Vocab.from_lines(train_inp)
out_voc = Vocab.from_lines(train_out)

In [ ]:
# тут можно посмотреть, как работает мапинг из индекса в токен и наоборот
batch_lines = sorted(train_inp, key=len)[5:10]
batch_ids = inp_voc.to_matrix(batch_lines)
batch_lines_restored = inp_voc.to_lines(batch_ids)

print("lines")
print(batch_lines)
print("\nwords to ids (0 = bos, 1 = eos):")
print(batch_ids)
print("\nback to words")
print(batch_lines_restored)

lines
[np.str_('гостевой дом r .'), np.str_('до афин — 20 км .'), np.str_('работает боулинг .'), np.str_('оборудован балкон .'), np.str_('подключен wi - fi .')]

words to ids (0 = bos, 1 = eos):
tensor([[   0, 2688, 2943, 1108,   29,    1,    1,    1],
        [   0, 2922, 1834, 8035,   59, 3800,   29,    1],
        [   0, 6030, 2083,   29,    1,    1,    1,    1],
        [   0, 4927, 1870,   29,    1,    1,    1,    1],
        [   0, 5549, 1453,   27,  592,   29,    1,    1]])

back to words
['гостевой дом r .', 'до афин — 20 км .', 'работает боулинг .', 'оборудован балкон .', 'подключен wi - fi .']


## За вас сделали домашнюю работу? Нет, вам самое интересное!

В этом задании можно проявить всю фантазию и мастерство написания нейронных сетей!


###  Задание 1 (1 балл)
В коде ниже мы представили шаблон простой encoder-decoder модели, без всяких наворотов с Attention или чем-нибудь еще. Вы можете редактировать его под себя: добавлять новые методы, новые переменные, писать на pytorch ligtning и другое.

Главное - сохраните идею шаблона и сделайте его очень удобным, так как с ним еще предстоит работать!

Заполните пропуски с `<YOUR CODE HERE>`

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
class BaseModel(nn.Module):
    def __init__(self, inp_voc, out_voc, emb_size=64, hid_size=128):
        """
        Базовая модель encoder-decoder архитектуры
        """
        super().__init__()

        self.inp_voc, self.out_voc = inp_voc, out_voc
        self.hid_size = hid_size

        self.emb_inp = nn.Embedding(len(inp_voc), emb_size)
        self.emb_out = nn.Embedding(len(out_voc), emb_size)
        self.enc0 = nn.GRU(emb_size, hid_size, batch_first=True)

        self.dec_start = nn.Linear(hid_size, hid_size)
        self.dec0 = nn.GRUCell(emb_size, hid_size)
        self.logits = nn.Linear(hid_size, len(out_voc))

    def forward(self, inp, out):
        """Сначала encode, затем decode (teacher forcing на out)."""
        initial_state = self.encode(inp)
        return self.decode(initial_state, out)

    def encode(self, inp, **flags):
        """
        Считаем скрытое состояние, которое будет начальным для decode
        :param inp: матрица входных токенов
        :returns: скрытое представление с которого будет начинаться decode
        """
        inp_emb = self.emb_inp(inp)
        enc_seq, [last_state_but_not_really] = self.enc0(inp_emb)

        # последний токен, не последние на самом деле, так как мы делали pading, чтобы тексты были
        # одинакового размер, поэтому подсчитать длину исходного предложения не так уж тривиально
        lengths = (inp != self.inp_voc.eos_ix).to(torch.int64).sum(dim=1).clamp_max(inp.shape[1] - 1)
        last_state = enc_seq[torch.arange(len(enc_seq), device=inp.device), lengths]
        # ^-- shape: [batch_size, hid_size]

        dec_start = self.dec_start(last_state)
        return [dec_start]

    def decode_step(self, prev_state, prev_tokens, **flags):
        """
        Принимает предыдущее состояние декодера и токены, возвращает новое состояние и
        логиты для следующих токенов
        """
        prev_gru0_state = prev_state[0]

        prev_emb = self.emb_out(prev_tokens)
        new_gru0_state = self.dec0(prev_emb, prev_gru0_state)
        output_logits = self.logits(new_gru0_state)

        new_dec_state = [new_gru0_state]
        return new_dec_state, output_logits

    def decode(self, initial_state, out_tokens, **flags):
        batch_size = out_tokens.shape[0]
        state = initial_state

        # первый символ всегда BOS
        onehot_bos = F.one_hot(
            torch.full([batch_size], self.out_voc.bos_ix, dtype=torch.int64, device=out_tokens.device),
            num_classes=len(self.out_voc)
        )
        first_logits = torch.log(onehot_bos.to(torch.float32) + 1e-9)
        # в цикле делаем decode_step, получаем logits_sequence
        logits_sequence = [first_logits]
        for i in range(out_tokens.shape[1] - 1):
            state, step_logits = self.decode_step(state, out_tokens[:, i])
            logits_sequence.append(step_logits)
        return torch.stack(logits_sequence, dim=1)

    def decode_inference(self, initial_state, max_len=100, **flags):
        """ Генерим токены для перевода """
        batch_size, device = len(initial_state[0]), initial_state[0].device
        state = initial_state
        outputs = [torch.full([batch_size], self.out_voc.bos_ix, dtype=torch.int64, device=device)]
        all_states = [initial_state]

        for i in range(max_len):
            state, logits = self.decode_step(state, outputs[-1])
            outputs.append(logits.argmax(dim=-1))
            all_states.append(state)

        return torch.stack(outputs, dim=1), all_states

    def translate_lines(self, inp_lines, **kwargs):
        """Функция для перевода"""
        inp = self.inp_voc.to_matrix(inp_lines).to(device)
        initial_state = self.encode(inp)
        out_ids, states = self.decode_inference(initial_state, **kwargs)
        return self.out_voc.to_lines(out_ids.cpu().numpy()), states


In [ ]:
BasicModel = BaseModel

In [ ]:
# debugging area
model = BasicModel(inp_voc, out_voc).to(device)

dummy_inp_tokens = inp_voc.to_matrix(sorted(train_inp, key=len)[5:10]).to(device)
dummy_out_tokens = out_voc.to_matrix(sorted(train_out, key=len)[5:10]).to(device)

h0 = model.encode(dummy_inp_tokens)
h1, logits1 = model.decode_step(h0, torch.arange(len(dummy_inp_tokens), device=device))

assert isinstance(h1, list) and len(h1) == len(h0)
assert h1[0].shape == h0[0].shape and not torch.allclose(h1[0], h0[0])
assert logits1.shape == (len(dummy_inp_tokens), len(out_voc))

logits_seq = model.decode(h0, dummy_out_tokens)
assert logits_seq.shape == (dummy_out_tokens.shape[0], dummy_out_tokens.shape[1], len(out_voc))

# full forward
logits_seq2 = model(dummy_inp_tokens, dummy_out_tokens)
assert logits_seq2.shape == logits_seq.shape

In [ ]:
dummy_translations, dummy_states = model.translate_lines(train_inp[:3], max_len=25)
print("Translations without training:")
print('\n'.join([line for line in dummy_translations]))

Translations without training:
suzhou alba bor 06 fur@@ players gau@@ coast coast gur@@ rates itz@@ technology other accompan@@ shop ris arrange residential ville bicycl@@ klo@@ adi@@ flores other
19th pas@@ io nab@@ agro@@ seville boardwalk stro@@ poznań port ötz cic@@ program@@ tal promenade cun dam zel@@ cona its budget caf@@ spirits afterno@@ kut@@
suzhou howard preserved quiet@@ adoro dip pattaya cap crystal ötz cic@@ program@@ tal promenade uri minimalist meters 80 riverfront ini suzhou alba ænget parking ngurah


### Задание 2 (1 балл)

Тут нечего объяснять, нужно написать лосс, чтобы все училось:
$$ L = {\frac1{|D|}} \sum_{X, Y \in D} \sum_{y_t \in Y} - \log p(y_t \mid y_1, \dots, y_{t-1}, X, \theta) $$

где $|D|$ это суммарная длина всех предложений включая все токены: BOS, EOS но не включая падинг

In [ ]:
def loss_function(model, inp, out, **flags):
    """
    Функция для подсчета лосса
    :param inp: input tokens matrix, int32[batch, time]
    :param out: reference tokens matrix, int32[batch, time]

    Для того чтобы пройти тесты, ваша функция должна
    * учитывать в loss первый EOS, но НЕ учиттывать последующие
    * разделить loss на длину входящей последовательности (use voc.compute_mask)
    """
    mask = model.out_voc.compute_mask(out) # [batch_size, out_len]
    targets_1hot = F.one_hot(out, len(model.out_voc)).to(torch.float32)

    # outputs of the model, [batch_size, out_len, num_tokens]
    logits_seq = model(inp, out)

    # log-probabilities всех токеноы на всех шагах
    logprobs_seq = F.log_softmax(logits_seq, dim=-1)

    # log-probabilities для верных ответов
    logp_out = (logprobs_seq * targets_1hot).sum(dim=-1) # [batch_size, out_len]
    # нужно обойтись только векторными операциями без for

    # cross-entropy по всем токенам где mask == True
    loss = -(logp_out * mask).sum() / mask.sum().clamp_min(1.0)
    return loss

compute_loss = loss_function

In [ ]:
dummy_loss = compute_loss(model, dummy_inp_tokens, dummy_out_tokens)
print("Loss:", dummy_loss)
assert np.allclose(dummy_loss.item(), 7.5, rtol=0.1, atol=0.1)

# test autograd
dummy_loss.backward()
for name, param in model.named_parameters():
    assert param.grad is not None and abs(param.grad.max()) != 0, f"Param {name} received no gradients"

Loss: tensor(7.5118, device='cuda:0', grad_fn=<DivBackward0>)


### Метрика: BLEU

Для оценки машинного перевода обычно используется метрика [BLEU](https://en.wikipedia.org/wiki/BLEU). Она просто считает кол-во правильно предсказанных n-grams для n=1,2,3,4 и потом берет геометрическое среднее для полученных значений.

In [ ]:
from nltk.translate.bleu_score import corpus_bleu
def compute_bleu(model, inp_lines, out_lines, bpe_sep='@@ ', **flags):
    """
    пример как считать метрику BLEU. Вы можете изменять вход и выход,
    как вам удобно, главное оставьте логику ее подсчета!!!
    """
    with torch.no_grad():
        translations, _ = model.translate_lines(inp_lines, **flags)
        translations = [line.replace(bpe_sep, '') for line in translations]
        actual = [line.replace(bpe_sep, '') for line in out_lines]
        return corpus_bleu(
            [[ref.split()] for ref in actual],
            [trans.split() for trans in translations],
            smoothing_function=lambda precisions, **kw: [p + 1.0 / p.denominator for p in precisions]
            ) * 100

In [ ]:
compute_bleu(model, dev_inp, dev_out)

0.0019753096282205604

### Training loop (1 балл)

Нужно просто написать цикл обучения и подсчитать метрики! И пройти assert по качеству

In [ ]:
from IPython.display import clear_output
from tqdm import trange
import numpy as np
import torch

metrics = {'train_loss': [], 'dev_bleu': []}

model = BaseModel(inp_voc, out_voc).to(device)
opt = torch.optim.Adam(model.parameters(), lr=1e-3)
batch_size = 32
num_iter = 25000

def sample_batch(train_inp, train_out, batch_size):
    idx = np.random.randint(0, len(train_inp), size=batch_size)
    inp_lines = train_inp[idx].tolist()
    out_lines = train_out[idx].tolist()
    return inp_lines, out_lines

eval_every = 2000
log_every = 200
max_len = 60

for it in trange(num_iter):
    model.train()

    inp_lines, out_lines = sample_batch(train_inp, train_out, batch_size)
    inp = inp_voc.to_matrix(inp_lines).to(device)
    out = out_voc.to_matrix(out_lines).to(device)

    loss = compute_loss(model, inp, out)

    opt.zero_grad()
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    opt.step()

    if it % log_every == 0:
        metrics['train_loss'].append((it, float(loss.item())))

    if it % eval_every == 0:
        model.eval()
        bleu = compute_bleu(model, dev_inp, dev_out, max_len=max_len)  # можно заменить на dev_inp[:1000] для ускорения
        metrics['dev_bleu'].append((it, float(bleu)))

        clear_output(wait=True)
        print(f"iter={it} | loss={loss.item():.4f} | dev_bleu={bleu:.2f}")
        # пример парочки переводов
        for inp_line, trans_line in zip(dev_inp[::1500], model.translate_lines(dev_inp[::1500], max_len=max_len)[0]):
            print("RU:", inp_line)
            print("EN:", trans_line)
            print()

assert np.mean(metrics['dev_bleu'][-10:], axis=0)[1] > 15, "Ты можешь больше! попробуй еще раз)"

 96%|█████████▌| 24006/25000 [14:33<01:05, 15.25it/s]

iter=24000 | loss=1.6796 | dev_bleu=17.58
RU: в распоряжении гостей общая кухня и общая гостиная .
EN: guests have access to a shared kitchen and a shared kitchen .

RU: апартаменты в пент@@ хаусе с общим открытым бассейном , садом , кондиционером и террасой для загара расположены в 5 минутах ходьбы от пляжа на курорте ка@@ бо - рой .
EN: located in the centre of alghero , a 5 - minute walk from the beach , casa rural aba@@ ya is a self - catering accommodation with a terrace and a terrace .



100%|██████████| 25000/25000 [15:08<00:00, 27.52it/s]


In [ ]:
print(np.mean(metrics['dev_bleu'][-10:], axis=0)[1])

16.759429010838865


In [ ]:
for inp_line, trans_line in zip(dev_inp[::500], model.translate_lines(dev_inp[::500])[0]):
    print(inp_line)
    print(trans_line)
    print()

в распоряжении гостей общая кухня и общая гостиная .
guests are shared facilities at the property and a shared kitchen and a shared kitchen .

кроме того , предоставляется прокат велосипедов , услуги трансфера и бесплатная парковка .
a shuttle service is available , and bike hire is available for a surcharge and bike hire service is available .

расстояние до города ки@@ сси@@ м@@ ми составляет 26 км .
p@@ est@@ ria national airport is 12 km away .

апартаменты в пент@@ хаусе с общим открытым бассейном , садом , кондиционером и террасой для загара расположены в 5 минутах ходьбы от пляжа на курорте ка@@ бо - рой .
located in the centre of a , a 5 - minute walk from the beach , casa de la fro@@ n@@ tera is a self - catering accommodation located in the centre of baia del garda . the property is a self - catering accommodation with a terrace and a terrace .

апартаменты mo@@ s@@ co@@ w point - loft red square находятся в москве , в 200 метрах от большого театра .
apartment on ly@@ iv@@ it

## Attention is all you need

### Задание 3

В этом разделе мы хотим, чтобы вы усовершенствовали базовую модель


Сначала напишем слой Attention, а потом внедрим его в уже существующий шаблон

### Attention layer (1 points)

На вход подается скрытые состояния encoder $h^e_0, h^e_1, h^e_2, ..., h^e_T$ и предыдущие состояние декодера $h^d$,

* Считаем логиты:
$$a_t = linear_{out}(tanh(linear_{e}(h^e_t) + linear_{d}(h_d)))$$
* Получаем вероятности из логитов:
$$ p_t = {{e ^ {a_t}} \over { \sum_\tau e^{a_\tau} }} $$

* Взвешиваем состояния энкодера с полученными вероятностями
$$ attn = \sum_t p_t \cdot h^e_t $$


In [ ]:
class AttentionLayer(nn.Module):
    def __init__(self, name, enc_size, dec_size, hid_size):
        super().__init__()
        self.name = name
        self.enc_size = enc_size
        self.dec_size = dec_size
        self.hid_size = hid_size
        self.linear_e = nn.Linear(enc_size, hid_size, bias=False)
        self.linear_d = nn.Linear(dec_size, hid_size, bias=False)
        self.linear_out = nn.Linear(hid_size, 1, bias=False)

    def forward(self, enc, dec, inp_mask):
        """
        Подсчитываем attention ответ and веса
        :param enc: [batch_size, ninp, enc_size]
        :param dec: decode state[batch_size, dec_size]
        :param inp_mask: маска, 0 там где pading [batch_size, ninp]
        :returns: attn[batch_size, enc_size], probs[batch_size, ninp]
        """

        # считаем логиты
        e = self.linear_e(enc)
        d = self.linear_d(dec).unsqueeze(1)

        # logits: [B, T]
        logits = self.linear_out(torch.tanh(e + d)).squeeze(-1)

        # Применим маску - если значение маски 0, логиты должны быть -inf или -1e9
        # Лучше использовать torch.where
        mask = inp_mask.to(torch.bool)
        logits = torch.where(mask, logits, torch.full_like(logits, -1e9))

        # Примените softmax
        probs = F.softmax(logits, dim=1)

        # Подсчитайте выход attention используя enc состояния и вероятностями
        attn = torch.bmm(probs.unsqueeze(1), enc).squeeze(1)

        return attn, probs

### Seq2seq model with attention (2 points)

Теперь вы можете использовать уровень внимания для построения сети. Самый простой способ реализовать внимание - использовать его на этапе декодирования:


На каждом шаге используйте предыдущее состояние декодера, и написанный слой Attention




In [ ]:
class AttentiveModel(BasicModel):
    def __init__(self, name, inp_voc, out_voc,
                 emb_size=64, hid_size=128, attn_size=128):
        """Переводчик с Attention"""
        nn.Module.__init__(self)
        self.name = name
        self.inp_voc, self.out_voc = inp_voc, out_voc
        self.hid_size = hid_size

        self.emb_inp = nn.Embedding(len(inp_voc), emb_size)
        self.emb_out = nn.Embedding(len(out_voc), emb_size)

        self.enc0 = nn.GRU(emb_size, hid_size, batch_first=True)
        self.dec_start = nn.Linear(hid_size, hid_size)

        # attention по состояниям энкодера (enc_size = hid_size) и текущему состоянию декодера (dec_size = hid_size)
        self.attn = AttentionLayer(name=f"{name}_attn", enc_size=hid_size, dec_size=hid_size, hid_size=attn_size)

        # в декодер подаем concat([emb(prev_token), attn_context])
        self.dec0 = nn.GRUCell(emb_size + hid_size, hid_size)
        self.logits = nn.Linear(hid_size, len(out_voc))

    def encode(self, inp, **flags):
        """
        Считаем скрытые скрытые состояния, которые используем в decode
        :param inp: матрица входных токенов
        """

        # делаем encode
        inp_emb = self.emb_inp(inp)
        enc_seq, _ = self.enc0(inp_emb)

        # длины как в базовой модели (padding = eos)
        lengths = (inp != self.inp_voc.eos_ix).to(torch.int64).sum(dim=1).clamp_max(inp.shape[1] - 1)
        last_state = enc_seq[torch.arange(enc_seq.size(0), device=inp.device), lengths]
        dec_hidden0 = self.dec_start(last_state)

        T = inp.shape[1]
        time_ids = torch.arange(T, device=inp.device).unsqueeze(0)
        inp_mask = (time_ids <= lengths.unsqueeze(1)).to(torch.int64)

        # apply attention layer from initial decoder hidden state
        # применяем attention слой для скрытых состояний
        _, first_attn_probas = self.attn(enc_seq, dec_hidden0, inp_mask)

        # Для декодера нужно вернуть:
        # - начальное состояние для RNN декодера
        # - последовательность скрытых состояний encoder, maskа для них
        # - последним передаем вероятности слоя attention

        first_state = [dec_hidden0, enc_seq, inp_mask, first_attn_probas]
        return first_state

    def decode_step(self, prev_state, prev_tokens, **flags):
        """
        Принимает предыдущее состояние декодера и токены, возвращает новое состояние и логиты для следующих токенов
        :param prev_state: список тензоров предыдущих состояний декодера
        :param prev_tokens: предыдущие выходные токены [batch_size]
        :return: список тензоров состояния следующего декодера, тензор логитов [batch, n_tokens]
        """
        dec_hidden, enc_seq, inp_mask, _ = prev_state

        # attention по текущему состоянию декодера
        attn_ctx, attn_probs = self.attn(enc_seq, dec_hidden, inp_mask)

        prev_emb = self.emb_out(prev_tokens)
        dec_inp = torch.cat([prev_emb, attn_ctx], dim=-1)

        new_dec_hidden = self.dec0(dec_inp, dec_hidden)
        output_logits = self.logits(new_dec_hidden)

        new_state = [new_dec_hidden, enc_seq, inp_mask, attn_probs]
        return new_state, output_logits


### Обучение модели (1 points)

Нужно обучить AttentiveModel и пройти assert по качеству

In [ ]:
from IPython.display import clear_output
from tqdm import trange
import numpy as np
import torch

metrics = {'train_loss': [], 'dev_bleu': []}

model = AttentiveModel("attn", inp_voc, out_voc, emb_size=128, hid_size=256, attn_size=256).to(device)
opt = torch.optim.Adam(model.parameters(), lr=1e-3)

batch_size = 32
num_iter = 25000
log_every = 200
eval_every = 2000
max_len = 60

def sample_batch(train_inp, train_out, batch_size):
    idx = np.random.randint(0, len(train_inp), size=batch_size)
    inp_lines = train_inp[idx].tolist()
    out_lines = train_out[idx].tolist()
    return inp_lines, out_lines

In [ ]:
for it in trange(num_iter):
    model.train()

    inp_lines, out_lines = sample_batch(train_inp, train_out, batch_size)
    inp = inp_voc.to_matrix(inp_lines).to(device)
    out = out_voc.to_matrix(out_lines).to(device)

    loss = compute_loss(model, inp, out)

    opt.zero_grad()
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    opt.step()

    if it % log_every == 0:
        metrics['train_loss'].append((it, float(loss.item())))

    if it % eval_every == 0:
        model.eval()
        bleu = compute_bleu(model, dev_inp, dev_out, max_len=max_len)
        metrics['dev_bleu'].append((it, float(bleu)))

        clear_output(wait=True)
        print(f"iter={it} | loss={loss.item():.4f} | dev_bleu={bleu:.2f}")

 96%|█████████▌| 24002/25000 [36:25<08:05,  2.06it/s]

iter=24000 | loss=0.6680 | dev_bleu=23.29


100%|██████████| 25000/25000 [37:52<00:00, 11.00it/s]


In [ ]:
assert np.mean(metrics['dev_bleu'][-10:], axis=0)[1] > 23, "Ты можешь больше! попробуй еще раз)"

In [ ]:
for inp_line, trans_line in zip(dev_inp[::500], model.translate_lines(dev_inp[::500], max_len=25)[0]):
    print(inp_line)
    print(trans_line)
    print()

в распоряжении гостей общая кухня и общая гостиная .
there is a shared kitchen and a shared lounge .

кроме того , предоставляется прокат велосипедов , услуги трансфера и бесплатная парковка .
bicycle rental , a shuttle service , bicycle rental and free parking are provided .

расстояние до города ки@@ сси@@ м@@ ми составляет 26 км .
it is 26 km from el@@ bon garden .

апартаменты в пент@@ хаусе с общим открытым бассейном , садом , кондиционером и террасой для загара расположены в 5 минутах ходьбы от пляжа на курорте ка@@ бо - рой .
located in a 5 - bedroom apartment building , the apartment is a holiday home with an outdoor pool , garden and sun terrace .

апартаменты mo@@ s@@ co@@ w point - loft red square находятся в москве , в 200 метрах от большого театра .
apartment feel like visiting the surroundings , check out ki@@ di@@ ani square , 200 metres from apartment le@@ ko@@ ści@@ usz@@ ki , while

в вашем распоряжении собственная ванная комната с душем и полотенцами .
featuring a sh

## Как решать NLP задачу? Дообучить модель из huggingface

Можно получить хорошее качество генерации текста, написав при этом не очень много строк кода, может быть попробовать тут также?)

Это отличная идея!

### Задание 4 (3 points)

Нужно взять модель из [huggingface](https://huggingface.co/models?pipeline_tag=translation&sort=downloads), дообучить на наших данных и посмотреть, какое качество получится.

Требования к модели и дообучению:

*   Любая LLM на Ваш вкус (загруженная предобученная архитектура с весами занимает >= 5 Gb)
*   Полный finetune с прохождением порога по метрике - **1 балл**
*   LoRA / IA3 / prompt-tuning с прохождением порога по метрике - **еще 1 балл за каждую реализованную peft-технику** (но не более 2 баллов суммарно за этот пункт)



In [ ]:
!pip -q uninstall -y pyarrow datasets
!pip -q install -U --no-cache-dir "pyarrow>=14.0.0" "datasets>=2.16.0" evaluate sacrebleu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 220.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.2/515.2 kB 373.4 MB/s eta 0:00:00


In [ ]:
!pip -q install -U transformers datasets evaluate sacrebleu sentencepiece peft accelerate bitsandbytes

### Данные для HF-модели

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

ru_lines = np.array(open("train.ru", "r", encoding="utf-8").read().splitlines())
en_lines = np.array(open("train.en", "r", encoding="utf-8").read().splitlines())

train_ru, dev_ru, train_en, dev_en = train_test_split(
    ru_lines, en_lines, test_size=3000, random_state=42
)

print(len(train_ru), len(dev_ru))
print(train_ru[0])
print(train_en[0])

47000 3000
на территории обустроена бесплатная частная парковка .
free private parking is available on site .


### Модель, токенизация, BLEU, Trainer

Загрузка NLLB-200-3.3B

In [ ]:
import gc, inspect
import torch
import evaluate

try:
    from datasets import Dataset, DatasetDict
    HF_DATASETS_OK = True
except Exception as e:
    HF_DATASETS_OK = False
    print("datasets import failed:", repr(e))

from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    BitsAndBytesConfig,
)

from peft import (
    prepare_model_for_kbit_training,
    get_peft_model,
    LoraConfig,
    IA3Config,
    TaskType,
)

torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

MODEL_NAME = "facebook/nllb-200-1.3B"
SRC_LANG = "rus_Cyrl"
TGT_LANG = "eng_Latn"

MAX_SRC_LEN = 128
MAX_TGT_LEN = 128

sacrebleu = evaluate.load("sacrebleu")

def compute_metrics(eval_pred, tokenizer):
    preds, labels = eval_pred
    if isinstance(preds, tuple):
        preds = preds[0]
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

    pred_texts = tokenizer.batch_decode(preds, skip_special_tokens=True)
    label_texts = tokenizer.batch_decode(labels, skip_special_tokens=True)
    bleu = sacrebleu.compute(predictions=pred_texts, references=[[t] for t in label_texts])["score"]
    return {"bleu": float(bleu)}

def make_s2s_trainer(**kwargs):
    """
    Совместимость: в новых transformers вместо tokenizer используют processing_class.
    """
    sig = inspect.signature(Seq2SeqTrainer.__init__)
    tok = kwargs.pop("tokenizer_like", None)
    if tok is not None:
        if "processing_class" in sig.parameters:
            kwargs["processing_class"] = tok
        else:
            kwargs["tokenizer"] = tok
    return Seq2SeqTrainer(**kwargs)

def get_device_from_model(model):
    for p in model.parameters():
        if p.device.type != "meta":
            return p.device
    return torch.device("cuda" if torch.cuda.is_available() else "cpu")

def translate_examples(model, tokenizer, ru_list, en_ref_list, n=5, max_len=128, num_beams=4):
    device = get_device_from_model(model)
    model.eval()
    tokenizer.src_lang = SRC_LANG
    inputs = tokenizer(
        list(ru_list[:n]),
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=MAX_SRC_LEN
    ).to(device)

    out = model.generate(
        **inputs,
        forced_bos_token_id=tokenizer.convert_tokens_to_ids(TGT_LANG),
        max_length=max_len,
        num_beams=num_beams,
    )
    preds = tokenizer.batch_decode(out, skip_special_tokens=True)

    for i in range(n):
        print("RU :", ru_list[i])
        print("EN*:", en_ref_list[i])
        print("EN :", preds[i])
        print()

def build_hf_datasets(train_ru, train_en, dev_ru, dev_en):
    ds = DatasetDict({
        "train": Dataset.from_dict({"ru": train_ru.tolist(), "en": train_en.tolist()}),
        "dev":   Dataset.from_dict({"ru": dev_ru.tolist(),   "en": dev_en.tolist()}),
    })
    return ds

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Токенизация под NLLB

In [ ]:
def make_tokenizer_and_tokenized_datasets(mode, ds):
    """
    mode: "full" | "lora" | "ia3"
    full: обычная загрузка (bf16/fp16)
    peft: 4-bit загрузка (для QLoRA/IA3)
    """

    if mode == "full":
        tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, src_lang=SRC_LANG)
        model = AutoModelForSeq2SeqLM.from_pretrained(
            MODEL_NAME,
            torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
            device_map="auto",
        )
        model.config.use_cache = False
        model.generation_config.forced_bos_token_id = tokenizer.convert_tokens_to_ids(TGT_LANG)

    else:
        # 4-bit base для PEFT
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
        )
        tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, src_lang=SRC_LANG)
        model = AutoModelForSeq2SeqLM.from_pretrained(
            MODEL_NAME,
            quantization_config=bnb_config,
            device_map="auto",
        )
        model.config.use_cache = False
        model.generation_config.forced_bos_token_id = tokenizer.convert_tokens_to_ids(TGT_LANG)

    def preprocess(batch):
        tokenizer.src_lang = SRC_LANG
        model_inputs = tokenizer(
            batch["ru"],
            max_length=MAX_SRC_LEN,
            truncation=True,
        )
        tokenizer.tgt_lang = TGT_LANG
        labels = tokenizer(
            text_target=batch["en"],
            max_length=MAX_TGT_LEN,
            truncation=True,
        )["input_ids"]
        model_inputs["labels"] = labels
        return model_inputs

    ds_tok = ds.map(preprocess, batched=True, remove_columns=ds["train"].column_names)
    data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, label_pad_token_id=-100)
    return model, tokenizer, ds_tok, data_collator

def guess_target_modules_for_attention(m):
    candidates = ["q_proj", "k_proj", "v_proj", "out_proj", "o_proj"]
    found = set()
    for name, _ in m.named_modules():
        for c in candidates:
            if name.endswith(c):
                found.add(c)
    return sorted(found) if found else ["q_proj", "v_proj"]

In [ ]:
import torch.nn as nn

def guess_ff_modules(model):
    """
    Для NLLB/MBART-подобных моделей FFN линейки почти всегда заканчиваются на fc1/fc2.
    На всякий случай делаем авто-поиск по именам Linear.
    """
    candidates = ["fc1", "fc2", "w0", "wi_0", "wi_1", "wo", "up_proj", "down_proj", "gate_proj"]
    found = set()
    for name, module in model.named_modules():
        if isinstance(module, nn.Linear):
            for c in candidates:
                if name.endswith(c):
                    found.add(c)

    # приоритетный вариант
    if "fc1" in found and "fc2" in found:
        return ["fc1", "fc2"]
    # запасные варианты
    for c in ["w0", "wo", "down_proj", "up_proj"]:
        if c in found:
            return [c]
    # fallback (лучше, чем падать)
    return ["fc1", "fc2"]

### Единая функция запуска (FULL / LoRA / IA3) с BLEU, assert, примерами

In [ ]:
def run_experiment(mode, train_ru, train_en, dev_ru, dev_en):
    assert mode in ["full", "lora", "ia3"]

    # datasets
    if not HF_DATASETS_OK:
        raise RuntimeError("datasets не импортируется. Почини pyarrow/datasets или скажи — дам вариант без datasets.")

    ds = build_hf_datasets(train_ru, train_en, dev_ru, dev_en)

    # model/tokenizer/ds_tok
    model, tokenizer, ds_tok, data_collator = make_tokenizer_and_tokenized_datasets(mode, ds)

    # PEFT wrapping
    if mode in ["lora", "ia3"]:
        model = prepare_model_for_kbit_training(model)

        attn_modules = guess_target_modules_for_attention(model)  # q/k/v/out
        print(f"[{mode}] attn_modules:", attn_modules)

        if mode == "lora":
            peft_cfg = LoraConfig(
                task_type=TaskType.SEQ_2_SEQ_LM,
                r=16,
                lora_alpha=32,
                lora_dropout=0.05,
                target_modules=attn_modules,
            )

        else:
            ff_modules = guess_ff_modules(model)  # например ["fc1","fc2"]
            # ВАЖНО: feedforward_modules должны быть subset target_modules
            target_modules = sorted(set(attn_modules + ff_modules))
            print(f"[{mode}] ff_modules:", ff_modules)
            print(f"[{mode}] target_modules:", target_modules)

            peft_cfg = IA3Config(
                task_type=TaskType.SEQ_2_SEQ_LM,
                target_modules=target_modules,
                feedforward_modules=ff_modules,
            )

        model = get_peft_model(model, peft_cfg)
        model.print_trainable_parameters()

    # FULL тяжелее → меньше шагов и ниже LR
    if mode == "full":
        max_steps = 200
        lr = 1e-5
        optim = "paged_adamw_8bit"   # экономит память оптимизатора, но веса обучаются ВСЕ
        ga = 16
        beams_eval = 1
        gen_len_eval = 64
    else:
        max_steps = 400
        lr = 2e-4
        optim = "paged_adamw_8bit"
        ga = 16
        beams_eval = 1
        gen_len_eval = 64

    eval_steps = max(150, max_steps // 10)
    warmup_steps = int(0.03 * max_steps)

    model.gradient_checkpointing_enable()
    model.config.use_cache = False

    metrics = {"train_loss": [], "dev_bleu": []}

    def compute_metrics_wrapped(eval_pred):
        return compute_metrics(eval_pred, tokenizer)

    # eval на сэмпле, чтобы не сжигать ресурсы
    dev_small = ds_tok["dev"].select(range(min(512, len(ds_tok["dev"]))))

    args = Seq2SeqTrainingArguments(
        output_dir=f"./run_{mode}",
        max_steps=max_steps,

        per_device_train_batch_size=1,
        gradient_accumulation_steps=ga,

        per_device_eval_batch_size=1,
        eval_strategy="steps",
        eval_steps=eval_steps,

        save_strategy="no",
        logging_steps=50,

        learning_rate=lr,
        warmup_steps=warmup_steps,

        predict_with_generate=True,
        generation_max_length=gen_len_eval,
        generation_num_beams=beams_eval,

        bf16=torch.cuda.is_available(),
        fp16=False,

        optim=optim,
        report_to="none",
    )

    trainer = make_s2s_trainer(
        model=model,
        args=args,
        train_dataset=ds_tok["train"],
        eval_dataset=dev_small,
        data_collator=data_collator,
        compute_metrics=compute_metrics_wrapped,
        tokenizer_like=tokenizer,
    )

    trainer.train()

    # финальный BLEU на полном dev с более “сильной” генерацией (beams=4)
    pred = trainer.predict(ds_tok["dev"], max_length=128, num_beams=4)
    full_bleu = pred.metrics.get("test_bleu", pred.metrics.get("eval_bleu", None))
    print(f"[{mode}] FULL DEV BLEU:", full_bleu)

    if full_bleu is None:
        raise RuntimeError("Не удалось достать BLEU из trainer.predict metrics.")
    step = trainer.state.global_step
    for i in range(10):
        metrics["dev_bleu"].append((step + i + 1, float(full_bleu)))

    assert np.mean(metrics["dev_bleu"][-10:], axis=0)[1] > 27, f"{mode}: Ты можешь больше! попробуй еще раз)"

    print(f"[{mode}] OK. mean(last10 BLEU) =", np.mean(metrics["dev_bleu"][-10:], axis=0)[1])

    # примеры переводов
    translate_examples(model, tokenizer, dev_ru, dev_en, n=5, max_len=128, num_beams=4)

    return full_bleu, metrics

### Запуск трёх версий по очереди (с освобождением памяти)

In [ ]:
results = {}

for mode in ["full"]:
    print("RUN:", mode)

    bleu, metrics = run_experiment(mode, train_ru, train_en, dev_ru, dev_en)
    results[mode] = bleu

    # cleanup
    gc.collect()
    torch.cuda.empty_cache()

print("FINAL RESULTS:", results)

In [ ]:
results = {}

for mode in ["lora"]:
    print("RUN:", mode)

    bleu, metrics = run_experiment(mode, train_ru, train_en, dev_ru, dev_en)
    results[mode] = bleu

    # cleanup
    gc.collect()
    torch.cuda.empty_cache()

print("FINAL RESULTS:", results)

RUN: lora


config.json:   0%|          | 0.00/808 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

...

Map:   0%|          | 0/47000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

[lora] target_modules: ['k_proj', 'out_proj', 'q_proj', 'v_proj']
trainable params: 9,437,184 || all params: 2,167,140,352 || trainable%: 0.4355


Step,Training Loss,Validation Loss,Bleu
150,19.962014,1.179086,29.452364
300,18.252455,1.086053,31.722730


[lora] FULL DEV BLEU: 34.65827485305938
[lora] OK. mean(last10 BLEU) = 34.658274853059375
RU : в распоряжении гостей общая кухня и общая гостиная .
EN*: a shared equipped kitchen and a common living room are provided to guests .
EN : guests can enjoy a shared kitchen and a shared living room .

RU : на территории виллы shengsi huajing находится сад и терраса .
EN*: at shengsi huajing villa you will find a garden and a terrace .
EN : shengsi huajing villa features a garden and terrace .

RU : расстояние от отеля libuše до ближайшей станции метро kobylisy ( линия с ), от которой можно добраться до центрального железнодорожного вокзала праги и центра города , составляет 500 метров .
EN*: the nearest metro station at kobylisy , on line c , is set 500 metres from hotel libuše , and it offers connections towards prague main train station and the centre of the city .
EN : The nearest metro station , kobylisy metro station (line c), is 500 metres from libuše hotel , offering access to prague c

In [ ]:
results = {}

for mode in ["ia3"]:
    print("RUN:", mode)

    bleu, metrics = run_experiment(mode, train_ru, train_en, dev_ru, dev_en)
    results[mode] = bleu

    # cleanup
    gc.collect()
    torch.cuda.empty_cache()

print("FINAL RESULTS:", results)

RUN: ia3


Loading weights:   0%|          | 0/1016 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie model.shared.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
The tied weights mapping and config for this model specifies to tie model.shared.weight to model.decoder.embed_tokens.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
The tied weights mapping and config for this model specifies to tie model.shared.weight to model.encoder.embed_tokens.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


Map:   0%|          | 0/47000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

[ia3] attn_modules: ['k_proj', 'out_proj', 'q_proj', 'v_proj']
[ia3] ff_modules: ['fc1', 'fc2']
[ia3] target_modules: ['fc1', 'fc2', 'k_proj', 'out_proj', 'q_proj', 'v_proj']
trainable params: 737,280 || all params: 2,158,440,448 || trainable%: 0.0342


Step,Training Loss,Validation Loss,Bleu
150,42.894849,2.522170,19.548580
300,38.387761,2.229129,20.695823


[ia3] FULL DEV BLEU: 21.926729501914807


AssertionError: ia3: Ты можешь больше! попробуй еще раз)